# Final Report - Preprocessing & Language Classification 

### STEP 1 ###


In [1]:
# This code is heavily adapted from the following resources  

# https://towardsdatascience.com/solving-a-simple-classification-problem-with-python-fruits-lovers-edition-d20ab6b071d2
# https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/
# https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24
# https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python
# https://www.kaggle.com/rcushen/topic-modelling-with-lsa-and-lda
# https://www.analyticsvidhya.com/blog/2018/10/stepwise-guide-topic-modeling-latent-semantic-analysis/

In [2]:
import urllib
import urllib.request
import zipfile
import os
import nltk
import glob
import re
import csv
import time
import random
import sys
from urllib.request import urlopen
import urllib.request, urllib.error
import pandas as pd
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras import layers, models, optimizers
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
nltk.download('stopwords')  
from nltk.corpus import stopwords 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import tree





Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abdulaziz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Load the repository list

In [3]:
with open('./data/TopStaredRepositories.csv', 'r') as f:
    reader = csv.reader(f)
    templist = list(reader)
repoheader=templist[0]
repolist=templist[1:]
# print(repoheader)

repos=[]
for line in repolist:
    repos.append({'username':line[0],'reponame':line[1],'description':line[2],
                  'updateDate':line[3],'language':line[4],'stars':line[5],'tags':line[6],'url':line[7]})
print(repos[1])

{'username': '1', 'reponame': 'twbs', 'description': 'bootstrap', 'updateDate': "['the', 'most', 'popular', 'html', ',', 'css', ',', 'and', 'javascript', 'framework', 'for', 'developing', 'responsive', ',', 'mobile', 'first', 'projects', 'on', 'the', 'web', '.']", 'language': '2017-06-24T15:40:21Z', 'stars': 'JavaScript', 'tags': '112k', 'url': "['javascript', 'css', 'html', 'bootstrap', 'jekyll', 'site', 'scss']"}


In [ ]:
### Read the content of the readme file 


### https://codereview.stackexchange.com/questions/186614/text-cleaning-script-producing-lowercase-words-with-minimal-punctuation

def cleaning(text):
    text = re.sub(r'\b(?:(?:https?|ftp)://)?\w[\w-]*(?:\.[\w-]+)+\S*', ' ', text.lower())
    words = re.findall(r'[a-z]+', text)
    return ' '.join(words)


count = 0
for repo in repos:
    url = "https://raw.githubusercontent.com"+"/"+repo['username']+"/"+repo['reponame']+"/master/README.md"
    try:
        print(url)
        conn = urlopen(url)
        repo['read_me'] = cleaning(conn.read().decode('utf-8'))
        count +=1
    except:
        try:
            url = "https://raw.githubusercontent.com"+"/"+repo['username']+"/"+repo['reponame']+"/master/readme.md"
            conn = urlopen(url)
            repo['read_me'] = cleaning(conn.read().decode('utf-8'))    
        except:
            print("Error in {}".format(repo['reponame']))
            repo['read_me'] = None
            continue


https://raw.githubusercontent.com/0/freeCodeCamp/master/README.md
Error in freeCodeCamp
https://raw.githubusercontent.com/1/twbs/master/README.md
Error in twbs
https://raw.githubusercontent.com/2/EbookFoundation/master/README.md
Error in EbookFoundation
https://raw.githubusercontent.com/3/facebook/master/README.md
Error in facebook
https://raw.githubusercontent.com/4/d3/master/README.md
Error in d3
https://raw.githubusercontent.com/5/getify/master/README.md
Error in getify
https://raw.githubusercontent.com/6/tensorflow/master/README.md
Error in tensorflow
https://raw.githubusercontent.com/7/sindresorhus/master/README.md
Error in sindresorhus
https://raw.githubusercontent.com/8/vuejs/master/README.md
Error in vuejs
https://raw.githubusercontent.com/9/angular/master/README.md
Error in angular
https://raw.githubusercontent.com/10/robbyrussell/master/README.md
Error in robbyrussell
https://raw.githubusercontent.com/11/airbnb/master/README.md
Error in airbnb
https://raw.githubusercontent.co

Error in tastejs
https://raw.githubusercontent.com/104/minimaxir/master/README.md
Error in minimaxir
https://raw.githubusercontent.com/105/scrapy/master/README.md
Error in scrapy
https://raw.githubusercontent.com/106/jashkenas/master/README.md
Error in jashkenas
https://raw.githubusercontent.com/107/harvesthq/master/README.md
Error in harvesthq
https://raw.githubusercontent.com/108/lukehoban/master/README.md
Error in lukehoban
https://raw.githubusercontent.com/109/Modernizr/master/README.md
Error in Modernizr
https://raw.githubusercontent.com/110/square/master/README.md
Error in square
https://raw.githubusercontent.com/111/select2/master/README.md
Error in select2
https://raw.githubusercontent.com/112/vsouza/master/README.md
Error in vsouza
https://raw.githubusercontent.com/113/adam-p/master/README.md
Error in adam-p
https://raw.githubusercontent.com/114/sahat/master/README.md
Error in sahat
https://raw.githubusercontent.com/115/enaqx/master/README.md
Error in enaqx
https://raw.githubu

Error in SamyPesse
https://raw.githubusercontent.com/204/verekia/master/README.md
Error in verekia
https://raw.githubusercontent.com/205/usablica/master/README.md
Error in usablica
https://raw.githubusercontent.com/206/nostra13/master/README.md
Error in nostra13
https://raw.githubusercontent.com/207/ZuzooVn/master/README.md
Error in ZuzooVn
https://raw.githubusercontent.com/208/MaximAbramchuck/master/README.md
Error in MaximAbramchuck
https://raw.githubusercontent.com/209/ajaxorg/master/README.md
Error in ajaxorg
https://raw.githubusercontent.com/210/postcss/master/README.md
Error in postcss
https://raw.githubusercontent.com/211/bower/master/README.md
Error in bower
https://raw.githubusercontent.com/212/VundleVim/master/README.md
Error in VundleVim
https://raw.githubusercontent.com/213/Tencent/master/README.md
Error in Tencent
https://raw.githubusercontent.com/214/defunkt/master/README.md
Error in defunkt
https://raw.githubusercontent.com/215/kahun/master/README.md
Error in kahun
https

Error in wg
https://raw.githubusercontent.com/305/ggreer/master/README.md
Error in ggreer
https://raw.githubusercontent.com/306/jlmakes/master/README.md
Error in jlmakes
https://raw.githubusercontent.com/307/jasmine/master/README.md
Error in jasmine
https://raw.githubusercontent.com/308/cheeriojs/master/README.md
Error in cheeriojs
https://raw.githubusercontent.com/309/bbatsov/master/README.md
Error in bbatsov
https://raw.githubusercontent.com/310/enyo/master/README.md
Error in enyo
https://raw.githubusercontent.com/311/CamDavidsonPilon/master/README.md
Error in CamDavidsonPilon
https://raw.githubusercontent.com/312/segmentio/master/README.md
Error in segmentio
https://raw.githubusercontent.com/313/RocketChat/master/README.md
Error in RocketChat
https://raw.githubusercontent.com/314/kripken/master/README.md
Error in kripken
https://raw.githubusercontent.com/315/ruby/master/README.md
Error in ruby
https://raw.githubusercontent.com/316/interagent/master/README.md
Error in interagent
http

Download and unzip the repositories, with sleep added to avoid putting too much pressure on the website.

Some repositories without a master branch will not be downloaded.

In [ ]:
print(count)

In [ ]:
# https://stackoverflow.com/questions/17839973/constructing-pandas-dataframe-from-values-in-variables-gives-valueerror-if-usi
df = pd.DataFrame.from_dict(repos)

In [ ]:
df

In [ ]:
df.to_csv(".data/Final_dataset.csv", sep='\t')

### STEP 2 ###
        ## Language Classification ##


In [3]:
dataset = pd.read_csv("./data/Final_dataset_readme_content.csv", sep='\t')

In [4]:
dataset.shape

(980, 10)

In [5]:
sum(pd.isnull(dataset['language']))

103

In [6]:
sum(pd.isna(dataset['language']))

103

In [7]:
# dataset[pd.isna(dataset['language'])] 

In [8]:
# Let's test it on a smal dataset 

# Used only the useful attributes 
columns = ['description', 'language', 'read_me', 'reponame', 'tags', 'username']
df = dataset[columns]

In [9]:
df[:2]

,description,language,read_me,reponame,tags,username
0,The https://freeCodeCamp.com open source codeb...,JavaScript,social banner build status pull requests welco...,freeCodeCamp,"nonprofits,certification,curriculum,react,node...",freeCodeCamp
1,"The most popular HTML, CSS, and JavaScript fra...",JavaScript,bootstrap slack bower version npm version buil...,bootstrap,"javascript,css,html,bootstrap,jekyll-site,scss",twbs


In [10]:
df['collection'] = df[['description','read_me', 'reponame', 'tags', 'username']].apply(lambda x: ' '.join(x.astype(str)), axis=1)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
new_df = pd.DataFrame()
new_df['language'] = df['language']
new_df['collection'] = df['collection']

In [12]:
# new_df.to_csv('./data/language_Dataset.csv')

In [13]:
new_df_lang = new_df[pd.notna(new_df['language'])]

In [14]:
len(new_df_lang[pd.isnull(new_df_lang['language'])]) == 0

True

In [15]:
len(new_df_lang)

877

In [16]:
new_df_lang.columns

Index(['language', 'collection'], dtype='object')

In [17]:
# Using Stemmer and Limitizer 

# Convert it to a numpy array
X = new_df_lang['collection'].values
y = new_df_lang['language'].values

documents = []

for sen in range(0, len(X)):  
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    document = document.split()

    document = [lmtzr.lemmatize(word) for word in document]
    document = ' '.join(document)

    documents.append(document)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, 
                            stop_words = stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

In [19]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [20]:
from sklearn.model_selection import train_test_split  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [21]:
from sklearn.ensemble import RandomForestClassifier

classifer = RandomForestClassifier(n_estimators = 1000, random_state=0)
classifer.fit(X_train, y_train)
y_pred = classifer.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.6363636363636364


In [22]:
classifer = LogisticRegression()
classifer.fit(X_train, y_train)
y_pred = classifer.predict(X_test)
print(accuracy_score(y_test, y_pred))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.5852272727272727


In [23]:
classifer = GaussianNB()
classifer.fit(X_train, y_train)
y_pred = classifer.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.5


In [24]:
classifer = svm.SVC(gamma='scale')
classifer.fit(X_train, y_train)
y_pred = classifer.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.4375


In [25]:
classifer = tree.DecisionTreeClassifier()
classifer.fit(X_train, y_train)
y_pred = classifer.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.5113636363636364


##

In [26]:
# Now let's split the dataset into training and testing datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(new_df_lang['collection'], new_df_lang['language'])

In [27]:
# Encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [28]:
# Feature Engineering 

count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')

count_vect.fit(new_df_lang['collection'])

xtrain_count = count_vect.fit_transform(train_x)
xvalid_count = count_vect.fit_transform(valid_x)

In [29]:
# TF-IDF Vectors 

# Word Level TF_IDF
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(new_df_lang['collection'])

xtrain_tfidf = tfidf_vect.transform(train_x)
xvalid_tfidf = tfidf_vect.transform(valid_x)

# ngram Level TF_IDF
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(new_df_lang['collection'])

xtrain_tfidf_ngram = tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram = tfidf_vect_ngram.transform(valid_x)

# Character Level tf_idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(new_df_lang['collection'])

xtrain_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(train_x)
xvalid_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(valid_x)

In [30]:
#  Utility 

def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    print(predictions)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)


In [31]:
                ## Logistic Regression ##
    
accuracy1 = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)

print("Logistic Regression, TF-IDF Vectors {}".format(accuracy1))

accuracy1 = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf)

print("Logistic Regression, TF-IDF-Ngram Vectors {}".format(accuracy1))

accuracy1 = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf)

print("Logistic Regression, TF-IDF Vectors {}".format(accuracy1))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[15 22 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 33 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 14 15 15 15 15 15 15 15 15 15 15 28
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 14
 15 15 15 14 15 15 14 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 14 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 14 15 15 15 15 15 14 15 15 28 15 15 15 15 15 15 15 15 15 15 15 15 15
 14 15 15 15 15 15 15 15 36 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 14 15 15 22 14 15 15 15 15 14 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15]
Logistic Regression, TF-IDF Vectors 0.004545454545454545
[15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 1

In [32]:
                ## Naive Bayes ##

# Naive Bayes on Word Level TF- IDF Vectors 
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y,
                xvalid_tfidf)

print("NB, World TF-IDF Vectors {}".format(accuracy))

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram,
                      train_y, xvalid_tfidf_ngram)
print("NB, World N-Gram Vectors {}".format(accuracy))

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars,
                      train_y, xvalid_tfidf_ngram_chars)
print("NB, Char Vectors {}".format(accuracy))


[15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15]
NB, World TF-IDF Vectors 0.004545454545454545
[15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 14 15 15 15 15 15 15 15 15 15 15 28
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 

In [33]:
               ## SVM ##

accuracy = train_model(svm.SVC(), xtrain_tfidf, train_y,
                xvalid_tfidf)

print("NB, World TF-IDF Vectors {}".format(accuracy))

accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram,
                      train_y, xvalid_tfidf_ngram)
print("NB, World N-Gram Vectors {}".format(accuracy))

accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram_chars,
                      train_y, xvalid_tfidf_ngram_chars)
print("NB, Char Vectors {}".format(accuracy))



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15]
NB, World TF-IDF Vectors 0.004545454545454545
[15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 

In [34]:
              ## D Tree ##

accuracy = train_model(tree.DecisionTreeClassifier(), xtrain_tfidf, train_y,
                xvalid_tfidf)

print("NB, World TF-IDF Vectors {}".format(accuracy))

accuracy = train_model(tree.DecisionTreeClassifier(), xtrain_tfidf_ngram,
                      train_y, xvalid_tfidf_ngram)
print("NB, World N-Gram Vectors {}".format(accuracy))

accuracy = train_model(tree.DecisionTreeClassifier(), xtrain_tfidf_ngram_chars,
                      train_y, xvalid_tfidf_ngram_chars)
print("NB, Char Vectors {}".format(accuracy))

[22 22 12 15 29 15 12 15 12 24 15 12 29 36 15  2 24  2 15 15 12 32  3 15
 14 15 15 15 11 28 15 33 15 15  2 28  4 24 15 32 32 15 28 22 15  4  5 33
 15 12 15 15  2  4 15 15 15  4 28 15 14 11 15 15 22 11 14 15 15  0  4 11
 15 35 15 15 15 15 15  2  4 15 15 15 22 15 15 15 15 15 15 14 15 15 24 14
  4 12 12 14  4 15 14 15 15  4 15 15 15  5 15  2 29 15 22 15 15 14 22 24
 24 15 11 29 12 33 14  5 15 15 35 12 35 15 33 32 33 15 15 12 15  2 15 35
 29 14 28 14 15 33 37 14 15 22  2 29 11 15 15 12 11 29 15 11 22 33 15 11
 14 15 15 15 15 28 15 14 36 15 15 12 22 28 15 28 35 29 33 15 15 22 29 15
  2 15 14  2 15 33 14  5 15 15 15 14 15 15 14 15 15 11 14 32 15 28 29 22
 14 29 15 15]
NB, World TF-IDF Vectors 0.004545454545454545
[15 22 29 15  5 29 15 15 33 24 21 15 22 36 15 13 29 28 15 37 15 15 35 12
 12 15 15 15 15 12  5 35 15 15  4 15 15 15 15 14 28  4 28 15 35 35  7 15
 14 12 15 12 12 28 15 15 32 13 15 15 14 15 15 15 15 11 15 15 15 11  4 22
 15  4 15 24  5 15 13 33 34 29 29 15 12 36 15  5 35 24  5 15 15 

In [35]:
       
## Random Forest 

accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y,
                xvalid_tfidf)

print("RF, Word TF-IDF Vectors {}".format(accuracy))

accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram, train_y,
                xvalid_tfidf_ngram)

print("RF, Word TF-IDF Vectors Ngram {}".format(accuracy))


accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram_chars, train_y,
                xvalid_tfidf_ngram_chars)

print("RF, Word TF-IDF Vectors Ngram - Char {}".format(accuracy))

[22 22 15 15 12 15 15 15 15 15 15 15 15 15 15  4 15 15 15 15 15 29 28 15
 14 15 15 15  2 28 15 22 15 15  4 15 15 15 15 15 15 15 15 22 15  4 15 15
 15 15 15 15  4 15 15 15  2 15 28 15 14 15 15 15 11 11 15 15 15 14 35 15
 15 15 35 15 15 15 15 22 15 15 33 15 22  4 15 15 15 15 11 15 15 15 11 14
 15  2 15 14 15 15 14  2 15  4 15 15 15 15 15 15 11 15  2 15 15 15 22 15
 15 15 11 15 14 22 14 15 15 15 15 15 15 15  2 15 22 15 15 15 15 15 12 15
 29 14 15 14 15  5 15 14 15 22 14 15 15 15 15 15 11 15  5 29 22 15 15 15
 14 11 14 15 15 28 15 14 11 15 15 15  4 15 15 15 15 15 22 15 15 22 15 15
  2 15 14 15 15 22 14 15 15 15 15 14 15 15 11 15 15 15 14 15 15 15 15 28
 15 29 15 15]
RF, Word TF-IDF Vectors 0.004545454545454545


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[15 22 15 15 15 15 15 15 33 15 15 15 15 12 15 15 15 28 15 15 15  4 15 15
 15 15 15 15 15 15 15 22 15 15 28 15 15  4 15 15  5 15 28 22 15 15 15 15
 15 15 15 15 28 28 15 15  5 15 15 15 14 15 15 15 15 15 15 15 15 14 14 28
 15 15 15 15 15 15 15 15 15 15 15 15 28 15 15 15 15 15 15 15 15 15 28 14
 15 15 15 14 22 15 28 15 15 15 15 15 15 15 15 28 29 15 15 15 15 15 22 15
 15 15 15 14 14 15 15 15 15 15 15 15 15 15 28 15 22 15 15 15 15 15 15 15
 15 14 12 15 15 15 15  2 15 12 15 29 29 15 15 15 15 15 15  5 15 15 15 15
 14 14 14 15 15 15 15 15 15 15 15 15 28 15 15 28 15 15 22 15 15 22 15 15
  4 15 14 15 15 22 15  4 15 15 15 14 15 15 28 15 15 15 14 15 15 15 15 15
 15 15 15 15]
RF, Word TF-IDF Vectors Ngram 0.004545454545454545


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[15  2 22 15 15 15 15 15 15  4 15 15  4 36 15 11 15  2  4 15 15  5 15 15
 14 15 15 15 28 28 15 22 15 15 15 28 15 15  2 15 15 15 28 22 15 29 15 22
 15 15 15 15  2 11 15 15  4 22 28  2 14 15 15 15 15 11 11 15 15 11 14 14
 28 15  2 15 15 15 15 22 14 15  4 15 33 15 15 15 15 15 15  4 15 15 15 14
  5 14 15 14 15  2 14 15 15 24 15 15 15 15 15 15  2 15 15 24 15  4 22 15
 15 14 11  4 15 15 14 15 15 15 15 15 15 15  2 24 15 15 15 15 15  2 15 15
  3 14 15 14 15 15 15 14 15  2 15  4 15 15 15 15 35 15  2  3 22 28 11 14
 14  3 15 15 15 28 15 15  2 15 15 15 15  2 15 22 15 15 22 15 15 33 15 15
 15 15 14 15 28 22 14 15 15 15 15 14 15 15 12 15 15 11 14 15 15 15 15 15
 15 29 22 15]
RF, Word TF-IDF Vectors Ngram - Char 0.013636363636363636


## Next Dataset ##

In [36]:
dataset = pd.read_csv("./data/Final_dataset_source_file_content.csv", sep='\t')

In [37]:
dataset.shape

(980, 10)

In [38]:
sum(pd.isnull(dataset['language']))

103

In [39]:
sum(pd.isna(dataset['language']))


103

In [40]:
dataset[:3]

,Unnamed: 0,username,reponame,url,updateDate,tags,stars,language,description,collections
0,0,freeCodeCamp,freeCodeCamp,https://github.com/freeCodeCamp/freeCodeCamp,2017-06-24T15:56:17Z,"nonprofits,certification,curriculum,react,node...",290k,JavaScript,The https://freeCodeCamp.com open source codeb...,name Managing Packages with npm dashedName man...
1,1,twbs,bootstrap,https://github.com/twbs/bootstrap,2017-06-24T15:40:21Z,"javascript,css,html,bootstrap,jekyll-site,scss",112k,JavaScript,"The most popular HTML, CSS, and JavaScript fra...",stylelint disable normalize css MIT License gi...
2,2,EbookFoundation,free-programming-books,https://github.com/EbookFoundation/free-progra...,2017-06-23T01:09:34Z,"education,list,books,resource",87.8k,NaN,Freely available programming books,NaN


In [41]:
# Used only the useful attributes 
columns = ['description', 'language', 'reponame', 'tags', 'username', 'collections']

In [42]:
df = dataset[columns]
df[:2]

,description,language,reponame,tags,username,collections
0,The https://freeCodeCamp.com open source codeb...,JavaScript,freeCodeCamp,"nonprofits,certification,curriculum,react,node...",freeCodeCamp,name Managing Packages with npm dashedName man...
1,"The most popular HTML, CSS, and JavaScript fra...",JavaScript,bootstrap,"javascript,css,html,bootstrap,jekyll-site,scss",twbs,stylelint disable normalize css MIT License gi...


In [43]:
df['collection'] = df[['description', 'reponame', 'tags', 'username','collections']].apply(lambda x: ' '.join(x.astype(str)), axis=1)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [44]:
new_df = pd.DataFrame()

In [45]:
new_df['language'] = df['language']
new_df['collection'] = df['collection']
new_df.to_csv("./data/language_Dataset_source_content.csv", sep='\t')

In [46]:
new_df[:3]

,language,collection
0,JavaScript,The https://freeCodeCamp.com open source codeb...
1,JavaScript,"The most popular HTML, CSS, and JavaScript fra..."
2,NaN,Freely available programming books free-progr...


In [47]:
new_df_lang = new_df[pd.notna(new_df['language'])]

In [48]:
len(new_df_lang[pd.isnull(new_df_lang['language'])]) == 0

True

In [49]:
# Using Stemmer and Limitizer 

# Convert it to a numpy array
X = new_df_lang['collection'].values
y = new_df_lang['language'].values

documents = []

for sen in range(0, len(X)):  
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    document = document.split()

    document = [lmtzr.lemmatize(word) for word in document]
    document = ' '.join(document)

    documents.append(document)

In [50]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, 
                            stop_words = stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

In [51]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [52]:
from sklearn.model_selection import train_test_split  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [53]:
from sklearn.ensemble import RandomForestClassifier

classifer = RandomForestClassifier(n_estimators = 1000, random_state=0)
classifer.fit(X_train, y_train)
y_pred = classifer.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.6988636363636364


In [54]:
classifer = LogisticRegression()
classifer.fit(X_train, y_train)
y_pred = classifer.predict(X_test)
print(accuracy_score(y_test, y_pred))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6761363636363636


In [55]:
classifer = GaussianNB()
classifer.fit(X_train, y_train)
y_pred = classifer.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.4943181818181818


In [56]:
classifer = svm.SVC(gamma='scale')
classifer.fit(X_train, y_train)
y_pred = classifer.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.4659090909090909


In [57]:
classifer = tree.DecisionTreeClassifier()
classifer.fit(X_train, y_train)
y_pred = classifer.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.5852272727272727


In [58]:
# Now let's split the dataset into training and testing datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(new_df_lang['collection'], new_df_lang['language'])

In [59]:
# Encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [60]:
# Feature Engineering 

count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')

count_vect.fit(new_df_lang['collection'])

xtrain_count = count_vect.fit_transform(train_x)
xvalid_count = count_vect.fit_transform(valid_x)

In [61]:
# TF-IDF Vectors 

# Word Level TF_IDF
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(new_df_lang['collection'])

xtrain_tfidf = tfidf_vect.transform(train_x)
xvalid_tfidf = tfidf_vect.transform(valid_x)

# ngram Level TF_IDF
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(new_df_lang['collection'])

xtrain_tfidf_ngram = tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram = tfidf_vect_ngram.transform(valid_x)

# Character Level tf_idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(new_df_lang['collection'])

xtrain_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(train_x)
xvalid_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(valid_x)

In [62]:
#  Utility 

def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    print(predictions)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [63]:
                ## Logistic Regression ##
    
accuracy1 = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)

print("Logistic Regression, TF-IDF Vectors {}".format(accuracy1))

accuracy1 = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf)

print("Logistic Regression, TF-IDF-Ngram Vectors {}".format(accuracy1))

accuracy1 = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf)

print("Logistic Regression, TF-IDF Vectors {}".format(accuracy1))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[10 11 11 11 11 11 11 11 10 11 11 11  7 11 11 11 11 11 11 24 18 11  7 11
 11  7 11 11 10 18 11 11 11 10 11 11 11 11 11 11 10 11 10 11 18 11 18 11
 11 11 11 18 18 11 11 18 11 11 11 11 11 18 24 11 10 11 11  0 11 11 11 10
 18 11 11 11 11 11 11 11 11 11 24 33 11 11 11 18 11 11 11 11 11 11 11 11
 24 11 26 11 11 11 11 11 11 24 11 10 11 11 11 18 11 11 11 11 10 30 20 10
 11 11 11 10 11 11 11 11 11 11 11 11 10 11 11 11 24 11 26 11 11 10 11 11
 11 10 11 11 11 11 11 11 11 18 11 33 18 11 11  0 11 11  7 11 11 11 11 11
 18 24 11 11 10 11 11 11 11 11 11 11 24 11 11 10 11 11 11 11 10 24 11  0
 11 11 18 11 11 11 11 11 10 10 11 11 11  7 11 11 18 24 11 11 11 11 11 11
 11 24 11 11]
Logistic Regression, TF-IDF Vectors 0.00909090909090909
[11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11

In [64]:
               ## Naive Bayes ##

# Naive Bayes on Word Level TF- IDF Vectors 
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y,
                xvalid_tfidf)

print("NB, World TF-IDF Vectors {}".format(accuracy))

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram,
                      train_y, xvalid_tfidf_ngram)
print("NB, World N-Gram Vectors {}".format(accuracy))

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars,
                      train_y, xvalid_tfidf_ngram_chars)
print("NB, Char Vectors {}".format(accuracy))

[10 11 11 11 11 11 11 11 10 11 11 11 11 11 11 11 11 11 11 24 18 11 11 11
 11 11 11 11 10 18 11 11 11 10 11 11 11 11 11 11 10 11 11 11 18 11 11 11
 11 11 11 11 18 11 11 18 11 11 11 11 11 18 11 11 10 11 11 11 11 11 11 10
 18 11 11 11 11 11 11 11 11 11 11 11 11 11 11 18 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 10 11 11 11 18 11 11 11 11 10 11 11 10
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 24 11 11 11 11 10 11 11
 11 10 11 11 11 11 11 11 11 18 11 11 18 11 11 11 11 11 11 11 11 11 11 11
 18 11 11 11 11 11 11 11 11 11 11 11 11 11 11 10 11 11 11 11 10 11 11 11
 11 11 18 11 11 11 11 11 10 10 11 11 11 11 11 11 18 24 11 11 11 11 11 11
 11 24 11 11]
NB, World TF-IDF Vectors 0.00909090909090909
[10 11 11 11 11 11 11 11 10 11 11 11 11 11 11 11 11 11 11 24 18 11 11 11
 11 11 11 11 10 18 11 11 11 10 11 11 11 11 11 11 10 11 11 11 18 10 30 11
 11 11 11 11 18 11 11 18 11 11 11 11 11 18 11 11 10 11 11 11 26 11 11 10
 18 11 11 11 11 11 11 11 11  7 24 11 10 11 11 18 11 11 11 11 11 1

In [65]:
               ## SVM ##

accuracy = train_model(svm.SVC(), xtrain_tfidf, train_y,
                xvalid_tfidf)

print("NB, World TF-IDF Vectors {}".format(accuracy))

accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram,
                      train_y, xvalid_tfidf_ngram)
print("NB, World N-Gram Vectors {}".format(accuracy))

accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram_chars,
                      train_y, xvalid_tfidf_ngram_chars)
print("NB, Char Vectors {}".format(accuracy))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11]
NB, World TF-IDF Vectors 0.00909090909090909
[11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 1

In [66]:
              ## D Tree ##

accuracy = train_model(tree.DecisionTreeClassifier(), xtrain_tfidf, train_y,
                xvalid_tfidf)

print("NB, World TF-IDF Vectors {}".format(accuracy))

accuracy = train_model(tree.DecisionTreeClassifier(), xtrain_tfidf_ngram,
                      train_y, xvalid_tfidf_ngram)
print("NB, World N-Gram Vectors {}".format(accuracy))

accuracy = train_model(tree.DecisionTreeClassifier(), xtrain_tfidf_ngram_chars,
                      train_y, xvalid_tfidf_ngram_chars)
print("NB, Char Vectors {}".format(accuracy))

[ 0  8 11 32  5 11 11 30 10 11 26  5 11 11 11  0  2 11 11 24 18 11 30  3
 11 11 11  2 10 18  1 11 11 10 11 11  6 11 32 11 10  8 10 11 18 11 11  3
  3  8 11 18 30 30  8 18 11  2  0  7 11 30 24 11 10 11 11  0 11 11 26 10
 30 11 30 11  3 26 11 24 11  7 11 33 11 11  8 18 11 11  3 11 11  3  3  8
 11 11 26 26 11 11  0 11 11 24 11 11 11  3 30 18 11 30 11 11 10 30 20 10
 11 11  8 26 15 32 11 11 11 20  8  4  2  8 11  4  8 33 26 11 33 10 10  7
  2 10 11  8 11 11 24 20  8 18 11 29 18 11 11  2 11 11  7  8 11  0 10  8
 18 29 20 11 10 26 11 11 20 11 11  8 13 11 11 10  8 11 11 26 10 24 20 30
  8 11 30  7 11 11 11  4 10 10  3 32  0 11 11 11 18  8 11 29 11  3 11  2
 11 24 26 11]
NB, World TF-IDF Vectors 0.00909090909090909
[ 7 33 11 11  7 11 11 32 10 11 24 11 29 11 11  2  2 11  3 24 26 11  1 11
  3  7 11 33 10 18 18  7 26 10 11 11 11 33 33 11 10 33 10 11 18 11 18 11
 11 33 11 18 30  1 30 33 11 11  0 15 11 30 33 11 10 11 11  2  8 32  8 10
 30 11 11 11  3  1 11  0 11  7 33 33 11 11 11 18 33 26 33 11 11 1

In [68]:
       
## Random Forest 

accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y,
                xvalid_tfidf)

print("RF, Word TF-IDF Vectors {}".format(accuracy))

accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram, train_y,
                xvalid_tfidf_ngram)

print("RF, Word TF-IDF Vectors Ngram {}".format(accuracy))


accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram_chars, train_y,
                xvalid_tfidf_ngram_chars)

print("RF, Word TF-IDF Vectors Ngram - Char {}".format(accuracy))



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[ 2 11 11 11 11 11 11 11 10 11  3 20 20 11 11 11  0 11 11 24 18 11 10 11
 11  7 11 11 10 30 26 11 11 10 11 11 11 11 11 11 10 26  8 24 18 11 30 11
 11 11 11 18 30 30 20 30 11 11 11 24 11 30 24 11 10 11 11  0 11 24 11 10
 30 11 11 11 11  7 11 24 11 11 24 33 11 11 11 18  2 24  8 11 11 11 11 11
 24 11 26 33 11  7  0 11 11 24 11  0  2 11 18 30 11  0 11 11 10 20 20 10
 11 11 11  2  8 11 11 11 11 11 29 29 10 20 11 11 16 11 26 11 11 10 10 11
 11 10 11 11 11 11 11 20 11 30 11 33 18  8 11  0 11 11  7 26 11  0 10 11
 18 24 11 11 10 11 11  8 20 11 11 26 24 11 11 10 26 11 11 18 10 24 11  0
 11 11 30 11 11 11 11 11 10 10 11 30  0 30 11 11 18 24 11 11 11 11 11 10
 11 24 11 11]
RF, Word TF-IDF Vectors 0.00909090909090909
[10 11 11 11 11 11  7 11  7 11 24 11  7 11 11  2 11 11 11 24 18 11  7 11
 20  7 11 11 10 18 18 11 11 10 31 11 11 10 17 11 10 33 10 11 18 10 30 11
 11 11 11 18 30 30 30 18 11 11 11 24 11 30 24 11 10 11 11  2 11 11 11 10
 18 11 11 11  3 26 11  0 11  7 24 33 11 11 11 18  2 11 33 11 11 11

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[10 11 11 11 11 11 11 11 10 11 18 24  7 10 11 11  2 11  8 11 18 11  2 11
 24  2 11  2 10 30 11 11 11 10 11 11 11 11 11 11 10 26 10 11 18 11  9 11
 11 11 11 26 30 11 11 30 11 11 11 24 11 30 24 11 10 11 11  0 11 26 11 10
 30 11 11 11  3 24 11 11 11  2 11 33 10 11 11 18  2  0 18 11 11 11 11 11
  0 11 26 11 11 18 24  8 11 24 11 11 11 11 11 11 11  2 11 11 10 30 20 10
 11 11 11 24 11 11 11 11 11 11 18 24  8 10  0 11 24 11 26 11 11 10 11 11
 11 10 11 24 11 11  1 20 11 30 11 26 18 11 11  0 11 11 11  3 11  0 11 26
 18 11 11 11 10 11 11 11 26 18 11 11 24 11 11 10 10 11 11 26 10 11 11 11
 11 11 30 11 11 11 11 11 10 10 11 11  0  3 11 11 30 24 18 26 11 11 11  3
 11 24 11 11]
RF, Word TF-IDF Vectors Ngram - Char 0.00909090909090909
